In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# read dataset from step 2
appended_data5 = pd.read_csv('C:/Users/faraz/OneDrive - Ultranav/DTU/Semester 4/02806 - Social data analysis & visualization/Project/data/data_prepare2.csv',index_col=0)

C:\Users\faraz\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
# reset index
appended_data5 = appended_data5.reset_index(drop=True)

In [4]:
# convert userId column to string type
appended_data5['userId'] = appended_data5['userId'].astype(str)

In [5]:
# convert datetime column to pandas datetime
appended_data5['datetime'] = pd.to_datetime(appended_data5['datetime'])

In [6]:
# check the type of each column
appended_data5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82404129 entries, 0 to 82404128
Data columns (total 10 columns):
 #   Column           Dtype         
---  ------           -----         
 0   longitude        float64       
 1   latitude         float64       
 2   sport            object        
 3   heart_rate       int64         
 4   gender           object        
 5   timestamp        int64         
 6   userId           object        
 7   datetime         datetime64[ns]
 8   date             object        
 9   Day of the Week  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 6.1+ GB


In [7]:
# choose column needed for data analysis
appended_data5 = appended_data5[['longitude','latitude','sport','heart_rate','gender','userId','datetime','date','Day of the Week']]

In [8]:
# drop observations with heart_rate eqauls zero
appended_data5 = appended_data5.loc[appended_data5['heart_rate'] != 0]

In [9]:
# grouping observations based on userId
appended_df_list = [d for _, d in appended_data5.groupby(['userId'])]

In [10]:
# vectorized haversine function for claculating distance based on latitude and longitude

def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [11]:
# make distance and speed column
new_df = []
for i in range(len(appended_df_list)):
    for j in appended_df_list[i]['date'].unique():
        data = appended_df_list[i].loc[appended_df_list[i]['date']==j]
        data['lat_0'] = np.nan
        data['long_0'] = np.nan
        data['speed'] = np.nan
        data['lat_0'] = data['latitude'].shift(+1)
        data['long_0'] = data['longitude'].shift(+1)  
        data.lat_0.fillna(data.iloc[0]['latitude'], inplace=True)
        data.long_0.fillna(data.iloc[0]['longitude'], inplace=True)
        data['datetime_0'] = np.nan
        data['datetime_0'] = data['datetime'].shift(+1)
        data.datetime_0.fillna(data.iloc[0]['datetime'], inplace=True)
        data['Distance'] = (haversine(data.long_0, data.lat_0, data.longitude, data.latitude))*1000
        time = (data['datetime']-data['datetime_0']).astype('timedelta64[s]')
        data['speed'] = data['Distance']/time
        data['speed'] = data['speed'].fillna(0)
        new_df.append(data)

C:\Users\faraz\AppData\Local\Temp/ipykernel_3572/1642643343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['lat_0'] = np.nan
C:\Users\faraz\AppData\Local\Temp/ipykernel_3572/1642643343.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['long_0'] = np.nan
C:\Users\faraz\AppData\Local\Temp/ipykernel_3572/1642643343.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [13]:
new_df[2000]

,longitude,latitude,sport,heart_rate,gender,userId,datetime,date,Day of the Week,lat_0,long_0,speed,datetime_0,Distance
4843750,8.285367,55.628167,run,94,male,1048979,2012-07-14 00:17:36,2012-07-14,Saturday,55.628167,8.285367,0.000000,2012-07-14 00:17:36,0.000000
4843756,8.285402,55.628262,run,102,male,1048979,2012-07-14 00:17:44,2012-07-14,Saturday,55.628167,8.285367,1.406054,2012-07-14 00:17:36,11.248429
4843760,8.285249,55.628346,run,105,male,1048979,2012-07-14 00:17:50,2012-07-14,Saturday,55.628262,8.285402,3.219723,2012-07-14 00:17:44,19.318338
4843771,8.284297,55.628477,run,110,male,1048979,2012-07-14 00:18:03,2012-07-14,Saturday,55.628346,8.285249,8.218340,2012-07-14 00:17:50,106.838426
4843776,8.283074,55.628450,run,109,male,1048979,2012-07-14 00:18:15,2012-07-14,Saturday,55.628477,8.284297,11.341579,2012-07-14 00:18:03,136.098953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4845087,8.287899,55.627175,run,152,male,1048979,2012-07-14 01:51:50,2012-07-14,Saturday,55.626770,8.288211,7.070384,2012-07-14 01:51:42,56.563074
4845088,8.286293,55.627287,run,154,male,1048979,2012-07-14 01:52:02,2012-07-14,Saturday,55.627175,8.287899,14.915274,2012-07-14 01:51:50,178.983287
4845089,8.285500,55.627427,run,154,male,1048979,2012-07-14 01:52:09,2012-07-14,Saturday,55.627287,8.286293,12.782416,2012-07-14 01:52:02,89.476912
4845090,8.285068,55.627602,run,145,male,1048979,2012-07-14 01:52:18,2012-07-14,Saturday,55.627427,8.285500,5.753765,2012-07-14 01:52:09,51.783881


In [14]:
# make the final dataset with concating of all dataframes in the list
final_data = pd.concat(new_df)

In [15]:
# choose column needed for data analysis
final_data = final_data[['longitude','latitude','sport','heart_rate','gender','userId','datetime','date','Day of the Week','Distance','speed']]

In [17]:
# save dataset in the local drive
final_data.to_csv('C:/Users/faraz/OneDrive - Ultranav/DTU/Semester 4/02806 - Social data analysis & visualization/Project/data/final_data.csv')